**Build a simple question-answering chatbot using a pre-trained language model.**

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

class SimpleQABot:
    def __init__(self, model_name="bert-large-uncased-whole-word-masking-finetuned-squad"):
        """
        Initialize the QA bot with a pre-trained model
        Args:
            model_name (str): Name of the pre-trained model to use
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    def answer_question(self, context, question):
        """
        Answer a question based on the given context
        Args:
            context (str): The text passage to search for answers
            question (str): The question to answer
        Returns:
            dict: Contains the answer text and confidence score
        """
        # Tokenize input
        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            return_tensors="pt"
        )

        # Get model output
        input_ids = inputs["input_ids"].tolist()[0]
        outputs = self.model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits)

        # Convert tokens to answer text
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids)
        answer = tokens[answer_start]

        # Combine subword tokens
        for i in range(answer_start + 1, answer_end + 1):
            if tokens[i].startswith("##"):
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]

        # Calculate confidence score
        start_score = torch.max(torch.softmax(outputs.start_logits, dim=1)).item()
        end_score = torch.max(torch.softmax(outputs.end_logits, dim=1)).item()
        confidence = (start_score + end_score) / 2

        return {
            "answer": answer,
            "confidence": confidence
        }

def main():
    # Initialize the chatbot
    bot = SimpleQABot()

    print("Simple QA Chatbot (type 'quit' to exit)")
    print("First, please provide some context text for the bot to use.")

    context = input("\nContext: ").strip()

    while True:
        # Get question from user
        question = input("\nQuestion: ").strip()

        if question.lower() == 'quit':
            break

        # Get and display answer
        try:
            result = bot.answer_question(context, question)
            print(f"\nAnswer: {result['answer']}")
            print(f"Confidence: {result['confidence']:.2%}")
        except Exception as e:
            print(f"Error: {str(e)}")

        print("\n" + "-"*50)

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Simple QA Chatbot (type 'quit' to exit)
First, please provide some context text for the bot to use.

Context: Machine learning models are computational systems that learn patterns from data. The main types include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning models learn from labeled training data to make predictions or classifications. Examples include linear regression, logistic regression, decision trees, and neural networks.  Deep learning is a subset of machine learning that uses artificial neural networks with multiple layers. These deep neural networks excel at tasks like image recognition, natural language processing, and speech recognition. Popular deep learning frameworks include TensorFlow, PyTorch, and Keras.  The training process for ML models involves feeding them data, calculating errors using loss functions, and adjusting model parameters through optimization algorithms like gradient descent. Models must be carefully evalua